# [Optimizing the model parameters](https://learn.microsoft.com/en-us/training/modules/intro-machine-learning-pytorch/6-optimization)

In [ ]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork()

## 1.Setting hyperparameters

- Number of epochs
> The number times the entire training dataset is pass through the network.
- Batch Size
> The number of data samples seen by the model in each eposh. Iterates are the number of batches needs to compete an epoch.
- Learning Rate
> The size of steps the model match as it searchs for best weights that will produce a higher model accuracy.    
> - Smaller values means the model will take a longer time to find the best weights    
> - Larger values may result in the model step over and misses the best weights 

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 2.Add an optimization loop

- The Train Loop
> iterate over the training dataset and try to converge to optimal parameters.   
- The Validation/test Loop
> iterate over the test dataset to check if model performance is improving.

### Add a loss function
- `nn.MSELoss` (Mean Square Error) used for regression tasks
- `nn.NLLLoss` (Negative Log Likelihood) used for classification
- `nn.CrossEntropyLoss` combines `nn.LogSoftmax` and `nn.NLLLoss`

In [ ]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimization pass

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Backpropagation
- optimizer.zero_grad()
> 역전파 단계를 실행하기 전에 각 파라미터들의 변화도(gradient)를 0으로 재설정
- loss.backward()
> 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도 계산
- optimizer.step()
> 변화도를 계산한 뒤에 `optimizer.step()`을 호출하여 역전파 단계에서 수집된 변화도로 매개변수 조정

## 3.Full implementation

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312400  [    0/60000]
loss: 2.302949  [ 6400/60000]
loss: 2.291866  [12800/60000]
loss: 2.283075  [19200/60000]
loss: 2.278938  [25600/60000]
loss: 2.260535  [32000/60000]
loss: 2.263993  [38400/60000]
loss: 2.245359  [44800/60000]
loss: 2.234975  [51200/60000]
loss: 2.238173  [57600/60000]
Test Error: 
 Accuracy: 41.2%, Avg loss: 0.034841 

Epoch 2
-------------------------------
loss: 2.217512  [    0/60000]
loss: 2.226894  [ 6400/60000]
loss: 2.184963  [12800/60000]
loss: 2.198829  [19200/60000]
loss: 2.196058  [25600/60000]
loss: 2.150803  [32000/60000]
loss: 2.178365  [38400/60000]
loss: 2.130422  [44800/60000]
loss: 2.101382  [51200/60000]
loss: 2.126919  [57600/60000]
Test Error: 
 Accuracy: 43.7%, Avg loss: 0.032724 

Epoch 3
-------------------------------
loss: 2.084064  [    0/60000]
loss: 2.102448  [ 6400/60000]
loss: 2.014615  [12800/60000]
loss: 2.048793  [19200/60000]
loss: 2.054461  [25600/60000]
loss: 1.982034  [32000/600

## 4.Saving Models

In [ ]:
torch.save(model.state_dict(), "data/model.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 5.Loading Models

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("data/model.pth"))

<All keys matched successfully>

### Predictions

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Sandal", Actual: "Ankle boot"
